In [1]:
import torch 

In [2]:
A = torch.Tensor(
    [[ 3,4,2],
     [10,2,1],
     [ 1,1,1]])
b = torch.Tensor([[5,15,1]]).T

In [3]:
c = b.clone()
c[0,0]=18
print(b)

#np.copy(b)    in numpy

tensor([[ 5.],
        [15.],
        [ 1.]])


In [21]:
a = list(range(6,20))
print(a)
print(a[5])
print(a[5:10])
a = torch.Tensor(a)
print(a[5:10].argmax())
print(a[5:10].argmax()+5)
print(a[a[5:10].argmax()+5])

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
11
[11, 12, 13, 14, 15]
tensor(4)
tensor(9)
tensor(15.)


## LU decomposition with partial pivoting (without scaling)

In [22]:
def PLU(A, verbose=False): 
    n = A.size()[0]  
    assert A.size()[1]==n #assuming A is square
    
    L = torch.eye(n)
    U = A.clone()
    P = torch.eye(n)
    
    for k in range(0,n):

        colk = U[k:n,k]
        if verbose: print("#{}: colk={}".format(k,colk))
        pivot = torch.argmax(colk) + k 

        if U[pivot,k]==0: 
            raise Exception("A is singular") 
        if pivot != k: #pivot is not in current row. swap!
            if verbose: print("#{}: swapping {}<->{}".format(k,k,pivot))
            t = P[k].clone()
            P[k] = P[pivot].clone()
            P[pivot] = t

            t = U[k,k:n].clone()
            U[k,k:n] = U[pivot,k:n].clone()
            U[pivot,k:n] = t

            t = L[k,0:k].clone()
            L[k,0:k] = L[pivot,0:k].clone()
            L[pivot,0:k] = t    
        if verbose: print("P=\n{}".format(P))
        if verbose: print("L=\n{}".format(L))
        if verbose: print("U=\n{}".format(U))
        if verbose: print("pivot:{}".format(U[k,k]))
        for i in range(k+1,n): 
            L[i,k] = U[i,k]/U[k,k]    
            if verbose: print("#{},{}: L[{},{}]={}={}/pivot".format(i,k,i,k,L[i,k],U[i,k]))
            
            U[i] -= U[k]*L[i,k]
#             for j in range(k,n):
#                 U[i,j] = U[i,j] - L[i,k]*U[k,j];

        if verbose: print("L=\n{}".format(L))
        if verbose: print("U=\n{}".format(U))

    return P,L,U

In [23]:
P,L,U = PLU(A, verbose=True)

#0: colk=tensor([ 3., 10.,  1.])
#0: swapping 0<->1
P=
tensor([[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.]])
L=
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
U=
tensor([[10.,  2.,  1.],
        [ 3.,  4.,  2.],
        [ 1.,  1.,  1.]])
pivot:10.0
#1,0: L[1,0]=0.30000001192092896=3.0/pivot
#2,0: L[2,0]=0.10000000149011612=1.0/pivot
L=
tensor([[1.0000, 0.0000, 0.0000],
        [0.3000, 1.0000, 0.0000],
        [0.1000, 0.0000, 1.0000]])
U=
tensor([[10.0000,  2.0000,  1.0000],
        [ 0.0000,  3.4000,  1.7000],
        [ 0.0000,  0.8000,  0.9000]])
#1: colk=tensor([3.4000, 0.8000])
P=
tensor([[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.]])
L=
tensor([[1.0000, 0.0000, 0.0000],
        [0.3000, 1.0000, 0.0000],
        [0.1000, 0.0000, 1.0000]])
U=
tensor([[10.0000,  2.0000,  1.0000],
        [ 0.0000,  3.4000,  1.7000],
        [ 0.0000,  0.8000,  0.9000]])
pivot:3.4000000953674316
#2,1: L[2,1]=0.23529411852359772=0.800000011920929/pivot
L=
t

In [24]:
print(A) 
print(P.T.mm(L).mm(U))

tensor([[ 3.,  4.,  2.],
        [10.,  2.,  1.],
        [ 1.,  1.,  1.]])
tensor([[ 3.,  4.,  2.],
        [10.,  2.,  1.],
        [ 1.,  1.,  1.]])


In [25]:
print(P.mm(A))
print(L.mm(U))

tensor([[10.,  2.,  1.],
        [ 3.,  4.,  2.],
        [ 1.,  1.,  1.]])
tensor([[10.,  2.,  1.],
        [ 3.,  4.,  2.],
        [ 1.,  1.,  1.]])


In [26]:
import time
M = torch.randn((150,150))

T = time.time()
P,L,U = PLU(M)
T = time.time() - T
print(T)

0.498272180557251


In [29]:
M_est = P.T.mm(L).mm(U)
E = M_est - M
print("mean absolute error: {}".format(M.abs().mean())) 
print("mean squared error: {}".format((M**2).mean())) 

mean absolute error: 0.7989431619644165
mean squared error: 1.0022796392440796


## Same thing using torch

In [27]:
A_LU, pivots = A.lu()
P, A_L, A_U = torch.lu_unpack(A_LU, pivots)
print(P)
print(A_L)
print(A_U)
print(P.T.mm(A_L).mm(A_U))

tensor([[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.3000, 1.0000, 0.0000],
        [0.1000, 0.2353, 1.0000]])
tensor([[10.0000,  2.0000,  1.0000],
        [ 0.0000,  3.4000,  1.7000],
        [ 0.0000,  0.0000,  0.5000]])
tensor([[ 3.,  4.,  2.],
        [10.,  2.,  1.],
        [ 1.,  1.,  1.]])


In [28]:
T = time.time()
M_LU, pivots = M.lu()
T = time.time()-T
print(T)
P, L, U = torch.lu_unpack(M_LU, pivots)
M_est = P.T.mm(L).mm(U)
E = M_est - M
print("mean absolute error: {}".format(M.abs().mean())) 
print("mean squared error: {}".format((M**2).mean())) 

0.001965761184692383
mean absolute error: 0.7989431619644165
mean squared error: 1.0022796392440796
